In [745]:
import tensorflow as tf
import numpy as np
ep=np.exp(-10) #a small number

In [169]:
def soft_max(Qhat): #only takes float type argument
    sm=tf.nn.softmax(Qhat)
    sess=tf.Session()
    Q=sess.run(sm)
    sess.close()
    return Q

In [171]:
def one_hot_matrix(labels, K):
    """
    Creates a matrix where the i-th row corresponds to the ith dimension and the jth column
                     corresponds to the jth bin. So if dimension i had a label j, then entry (i,j) 
                     will be 1. 
                     
    Arguments:
    labels -- vector containing the bin label from function bin_b
    K -- number of bin
    """
    b = labels.flatten()
    
    # Use tf.one_hot, be careful with the axis
    one_hot_matrix = tf.one_hot(b,K,axis=1)
    sess = tf.Session()
    one_hot = sess.run(one_hot_matrix)
    sess.close()
    
    return one_hot

In [487]:
def cdf_piecewise_linear(xB,K,Q): #inputs: xB, matrix Q =(dim_xB,K) computed from xA, number of bins K
    print(xB)
    Q=soft_max(Q) #pdf
    print(Q)
    dim=np.shape(xB)[0] #get the dimension of xB
    b=np.floor(xB*K) #get the bin number of xB (dim,1)
    print(b)
    Qindex=np.pad([np.array(range(0,K))],((0,dim-1),(0,0)),"edge") #create a matrix outputting Q's index 
    part=(Qindex<b)*Q #output the part that is inside the sum of Eq(13), (dim,K)
    #print(cdfpart)
    proj=one_hot_matrix(b,K) #create a mask according to label b, (dim,K)
    cdftotal=np.sum(part+Q*proj*(K*xB-b),axis=1,keepdims=True) #output the cdf (dim,1)
    return cdftotal

In [486]:
#test cdf_piecewise_linear(np.ones((3,1)),5,np.random.randint(100,size=(3,5))/70) 

In [684]:
def normalize_V(V,Wn): #inputs: V =(dim_xB,K+1) and W =(dim_xB,K) computed and normalized from xA
    (dim,K)=np.shape(Wn) #get the dimension and number of bins
    expV=np.exp(V) #(dim,K+1)
    Vdeno=np.sum(0.5*(expV[:,0:K]+expV[:,1:K+1])*Wn,axis=1,keepdims=True) #(dim,1)   
    return np.true_divide(expV,Vdeno)

In [308]:
def normalize_W(W):
    return soft_max(W)

In [763]:
def bin_b_quad(xB,Wn):
    #print(xB)
    #print(Wn)
    (dim,K)=np.shape(Wn)
    Wcum=np.cumsum(Wn,axis=1) #get the cdf in each dimension of W
    mask=((xB-Wcum)>ep) #to avoid the precision issue, compare to a small number epsilon instead of zero
    Wtrun=mask*Wcum 
    b=np.reshape(np.argmax(Wtrun,axis=1),(dim,1))
    b=b+np.reshape(np.amax(Wtrun,axis=1)>0,(dim,1))*np.ones((dim,1))
    return b

In [769]:
def lerp(xB,Wn,Vn):
    (dim,K)=np.shape(Wn)
    b=bin_b_quad(xB,Wn)
    a=alpha(xB,Wn,b)
    #print(b)
    #print(a)
    proj=one_hot_matrix(b,K) #create a mask according to label b, (dim,K)
    #print(proj)
    diff= np.sum((-Vn[:,0:K]+Vn[:,1:K+1])*proj,axis=1,keepdims=True) #(dim,1)
    #print(diff)
    return a*(diff)+np.sum(Vn[:,0:K]*proj,axis=1,keepdims=True)

In [812]:
def cdf_piecewise_quad(xB,Wn,Vn):
    (dim,K)=np.shape(Wn)
    b=bin_b_quad(xB,Wn)
    #print(b)
    a=alpha(xB,Wn,b)
    #print(a)
    proj=one_hot_matrix(b,K) #create a mask according to label b, (dim,K)
    Wb=np.reshape(np.sum(Wn*proj,axis=1),(dim,1))
    
    Vb=np.sum(Vn[:,0:K]*proj,axis=1,keepdims=True)
    Vbplusone=np.sum(Vn[:,1:K+1]*proj,axis=1,keepdims=True)

    Windex=np.pad([np.array(range(0,K))],((0,dim-1),(0,0)),"edge") #create a matrix outputting W's indices
    part=(Windex<b)*(0.5*(Vn[:,0:K]+Vn[:,1:K+1])*Wn)
    constant=np.sum(part,axis=1,keepdims=True)
    #print(constant)
    
    return (a*a*0.5*(Vbplusone-Vb)+a*Vb)*Wb+constant

In [819]:
xB=np.random.rand(3,1)
print(xB)
Wn1=normalize_W(np.random.rand(3,5))
print(Wn1)
Vn1=normalize_V(np.random.rand(3,6),Wn1) #test
print(Vn1)
cdf_piecewise_quad(xB,Wn1,Vn1)

[[0.72339427]
 [0.53311484]
 [0.4246731 ]]
[[0.16348814 0.18076118 0.16805345 0.24745824 0.24023898]
 [0.1090726  0.27131492 0.22294379 0.12019002 0.27647868]
 [0.17452633 0.20760136 0.2192552  0.15584383 0.24277328]]
[[0.69464583 0.79815814 1.26264863 0.93450928 1.14257598 0.93951138]
 [0.83745868 0.75781313 0.65567189 1.54319449 1.21334771 1.03263934]
 [0.68909242 0.62649155 1.02077434 1.00542867 1.39238143 1.12223574]]


array([[0.70890496],
       [0.42531783],
       [0.32915525]])